In [12]:
from netCDF4 import Dataset, num2date, date2num
import csv
import glob
import xarray as xr 
import seaborn as sns
from wrf import (to_np, getvar, smooth2d, get_cartopy, cartopy_xlim, vertcross,
                 cartopy_ylim, interplevel, latlon_coords, uvmet, CoordPair,
                ALL_TIMES)
from matplotlib.dates import MonthLocator
import datetime
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import cartopy.feature as cfeature
import cartopy.crs as ccrs
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patheffects as PathEffects
import matplotlib.ticker as mticker
from matplotlib.cm import get_cmap
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER


In [13]:
def write_xarray_to_netcdf(xarray_array, output_path,mode='w', format='NETCDF4', group=None, engine=None,
                           encoding=None):
    """writes and xarray in a netcdf format outputfile
    Uses the xarray typical for wrf-python. The projection objects are transformed into strings
    to be able to use them as netcdf attributes
    :param xarray_array: xarray.DataArray
    :param output_path: str
    :param format: 'NETCDF4', 'NETCDF4_CLASSIC', 'NETCDF3_64BIT' or 'NETCDF3_CLASSIC'
                    default: 'NETCDF4'
    :param group: str, default None
    :param engine: 'netcdf4', 'scipy' or 'h5netcdf'
    :param encoding: dict, default: None
    """
    xarray_array_out = xarray_array.copy(deep=True)
    # coordinates are extracted from variable
    del xarray_array_out.attrs['coordinates']
    # wrf-python projection object cannot be processed
    xarray_array_out.attrs['projection'] = str(xarray_array_out.attrs['projection'])

    xarray_array_out.to_netcdf(path=output_path, mode=mode, format=format, group=group,
                               engine=engine,
                               encoding=encoding)

**Load stuff As needed from scratch or saved data**

In [14]:
Saved_Data_Path =  r'/glade/u/home/mwarms/SavedData/'
SNOWIE_Data_Path = r'/glade/u/home/mwarms/SNOWIE_Seasonal_Data/wrfout_d02'

# Load terrain & projection info
dummy_file = Dataset(SNOWIE_Data_Path + '/wrfout_d02_2016-10-01_00:00:00')
Ter_dom = getvar(dummy_file, "ter")
cartopy_dom = getvar(dummy_file, "ter")
cart_proj_dom = get_cartopy(cartopy_dom)

lat_min = 43
lat_max = 46
lon_min = -117
lon_max = -111

mask_lon = (cartopy_dom.XLONG >= lon_min) & (cartopy_dom.XLONG <= lon_max)
mask_lat = (cartopy_dom.XLAT >= lat_min) & (cartopy_dom.XLAT <= lat_max)

cartopy_aoi = cartopy_dom.where(mask_lon & mask_lat, drop=True)
cart_proj_aoi = get_cartopy(cartopy_aoi)

Ter_aoi = Ter_dom.where(mask_lon & mask_lat, drop=True)

# Load SNOWNC
SNOWNC_snowie = xr.open_dataarray(Saved_Data_Path + '/SNOWNC_snowie.nc')
SNOWNC_snowie = SNOWNC_snowie.where(mask_lon & mask_lat, drop=True)

# # Scratch Paths for QCLOUD by month
# Oct_Path =  r'/glade/scratch/mwarms/temp/\QCLOUD_snowie_oct.nc'
# Nov_Path =  r'/glade/scratch/mwarms/temp/\QCLOUD_snowie_nov.nc'
# Dec_Path =  r'/glade/scratch/mwarms/temp/\QCLOUD_snowie_dec.nc'
# Jan_Path =  r'/glade/scratch/mwarms/temp/\QCLOUD_snowie_jan.nc'
# Feb_Path =  r'/glade/scratch/mwarms/temp/\QCLOUD_snowie_feb.nc'
# Mar_Path =  r'/glade/scratch/mwarms/temp/\QCLOUD_snowie_mar.nc'
# Apr_Path =  r'/glade/scratch/mwarms/temp/\QCLOUD_snowie_apr.nc'

# # Load QCLOUDs
# QCLOUD_snowie_oct = xr.open_dataarray(Oct_Path)
# QCLOUD_snowie_nov = xr.open_dataarray(Nov_Path)
# QCLOUD_snowie_dec = xr.open_dataarray(Dec_Path)
# QCLOUD_snowie_jan = xr.open_dataarray(Jan_Path)
# QCLOUD_snowie_feb = xr.open_dataarray(Feb_Path)
# QCLOUD_snowie_mar = xr.open_dataarray(Mar_Path)
# QCLOUD_snowie_apr = xr.open_dataarray(Apr_Path)

# Delete unnecessary files to save space
del dummy_file, Ter_dom, mask_lon, mask_lat, lat_min, lat_max, cartopy_dom, cart_proj_dom
# del Oct_Path, Nov_Path, Dec_Path, Jan_Path, Feb_Path, Mar_Path, Apr_Path

In [16]:
# write_xarray_to_netcdf(SNOWNC_snowie, Saved_Data_Path + "SNOWNC_snowie_oct.nc", 
                       # mode = 'w', format = 'NETCDF4', group=None, engine=None, encoding=None)

**Find Events**

*Daily Averaging, Entire Grid & at Elevation Bands*

In [ ]:
SNOWNC_snowie_dailymean = SNOWNC_snowie.resample(Time='1D').last()

SNOWNC_spatial_avg_dailymean = np.zeros(212)
SNOWNC_spatial_avg_dailymean_a1500 = np.zeros(212)
SNOWNC_spatial_avg_dailymean_a2000 = np.zeros(212)
SNOWNC_spatial_avg_dailymean_a2500 = np.zeros(212)

for i in range (0,212): 
    SNOWNC_spatial_avg_dailymean[i] = SNOWNC_snowie_dailymean[i,:,:].mean() 
    SNOWNC_spatial_avg_dailymean_a1500[i] = SNOWNC_snowie_dailymean.where(Ter_aoi >= 1500)[i,:,:].mean()
    SNOWNC_spatial_avg_dailymean_a2000[i] = SNOWNC_snowie_dailymean.where(Ter_aoi >= 2000)[i,:,:].mean()    
    SNOWNC_spatial_avg_dailymean_a2500[i] = SNOWNC_snowie_dailymean.where(Ter_aoi >= 2500)[i,:,:].mean()  

*Turn Accumulation Into Snowfall Amounts*

In [ ]:
SNOWNC_snowie_dailymean_notaccum = SNOWNC_snowie_dailymean * 0
SNOWNC_snowie_notaccum = SNOWNC_snowie * 0

for i in range(0,len(SNOWNC_snowie_dailymean)):
    if i == 0:
        SNOWNC_snowie_dailymean_notaccum[i,:,:] =  SNOWNC_snowie_dailymean[i,:,:]
    else:
        SNOWNC_snowie_dailymean_notaccum[i,:,:] =  SNOWNC_snowie_dailymean[i,:,:] - SNOWNC_snowie_dailymean[i-1,:,:]
        
for i in range(0,len(SNOWNC_snowie)):
    if i == 0:
        SNOWNC_snowie_notaccum[i,:,:] =  SNOWNC_snowie[i,:,:]
    else:
        SNOWNC_snowie_notaccum[i,:,:] =  SNOWNC_snowie[i,:,:] - SNOWNC_snowie[i-1,:,:]

*Daily & Hourly Accumulation at Elevation Bands*

In [ ]:
SNOWNC_snowie_dailymean_notaccum_a1500 = SNOWNC_snowie_dailymean_notaccum.where(Ter_aoi > 1500)
SNOWNC_snowie_dailymean_notaccum_a2000 = SNOWNC_snowie_dailymean_notaccum.where(Ter_aoi > 2000)
SNOWNC_snowie_dailymean_notaccum_a2500 = SNOWNC_snowie_dailymean_notaccum.where(Ter_aoi > 2500)

SNOWNC_snowie_notaccum_a1500 = SNOWNC_snowie_notaccum.where(Ter_aoi > 1500)
SNOWNC_snowie_notaccum_a2000 = SNOWNC_snowie_notaccum.where(Ter_aoi > 2000)
SNOWNC_snowie_notaccum_a2500 = SNOWNC_snowie_notaccum.where(Ter_aoi > 2500)

*Average for all grid cells*

In [ ]:
SNOWNC_snowie_dailymean_notaccum_avg = np.zeros(212)
SNOWNC_snowie_dailymean_notaccum_avg_a1500 = np.zeros(212)
SNOWNC_snowie_dailymean_notaccum_avg_a2000 = np.zeros(212)
SNOWNC_snowie_dailymean_notaccum_avg_a2500 = np.zeros(212)

SNOWNC_snowie_notaccum_avg_a1500 = np.zeros(5088)
SNOWNC_snowie_notaccum_avg_a2000 = np.zeros(5088)
SNOWNC_snowie_notaccum_avg_a2500 = np.zeros(5088)
SNOWNC_snowie_notaccum_avg = np.zeros(5088)

for i in range (0,212):
    SNOWNC_snowie_dailymean_notaccum_avg[i] = SNOWNC_snowie_dailymean_notaccum[i,:,:].mean()
    SNOWNC_snowie_dailymean_notaccum_avg_a1500[i] = SNOWNC_snowie_dailymean_notaccum_a1500[i,:,:].mean()    
    SNOWNC_snowie_dailymean_notaccum_avg_a2000[i] = SNOWNC_snowie_dailymean_notaccum_a2000[i,:,:].mean()    
    SNOWNC_snowie_dailymean_notaccum_avg_a2500[i] = SNOWNC_snowie_dailymean_notaccum_a2500[i,:,:].mean()    

for i in range (0,5088):
    SNOWNC_snowie_notaccum_avg[i] = SNOWNC_snowie_notaccum[i,:,:].mean()
    SNOWNC_snowie_notaccum_avg_a1500[i] = SNOWNC_snowie_notaccum_a1500[i,:,:].mean()
    SNOWNC_snowie_notaccum_avg_a2000[i] = SNOWNC_snowie_notaccum_a2000[i,:,:].mean()
    SNOWNC_snowie_notaccum_avg_a2500[i] = SNOWNC_snowie_notaccum_a2500[i,:,:].mean()

*Find Events*

In [ ]:
SnowfallBelowThreshold = np.zeros(5088)
SnowfallEventNumbered = np.zeros(5088)
EventLengths = []

SnowfallThreshold = 0.025 # mm 
# SnowfallThreshold = 0.1 # mm 

for i in range(0,len(SnowfallBelowThreshold)):
    if SNOWNC_snowie_notaccum_avg[i] < SnowfallThreshold:
        SnowfallBelowThreshold[i] = 0
    else:
        SnowfallBelowThreshold[i] = SNOWNC_snowie_notaccum_avg[i]

EventTimeThreshold = 3 # hours
n = 1 # event counter
x = 1 # event length
j = 1 # hour counter

for i in range(0,len(SnowfallBelowThreshold)):
    j = 1
    if SnowfallBelowThreshold[i] == 0:
        if x > 1:
            x += 1
            SnowfallEventNumbered[i] = n 
            continue
        else:    
            continue
    else:     
        SnowfallEventNumbered[i] = n 
        while (j <= EventTimeThreshold and i+j < 5088):
            if SnowfallBelowThreshold[i+j] > 0:
                x += 1
                break
            if j == EventTimeThreshold:
                n += 1
                EventLengths.append(x)
                x = 1
            j += 1

*Other Useful Variables*

In [ ]:
MonthlyLastTimestep = [0, 744, 1464, 2208, 2952, 3624, 4368, 5088]
Months = ["Oct", "Nov", "Dec", "Jan", "Feb", "Mar", "Apr"]

EventTimesStart = []
EventTimesEnd = []
j = -1

for i in range(0,len(SnowfallEventNumbered)):
    if i > j:
        if SnowfallEventNumbered[i] > 0:
            EventTimesStart.append(i)
            for j in range(i,len(SnowfallEventNumbered)):
                if SnowfallEventNumbered[j] == 0:
                    EventTimesEnd.append(j)
                    break

EventPrecip = np.zeros(len(EventLengths))

for i in range(0,len(EventPrecip)):
    EventPrecip[i] = sum(SnowfallBelowThreshold[EventTimesStart[i]:EventTimesEnd[i]])
    
MonthlyEvents = np.zeros(7)

TotalEvents = max(SnowfallEventNumbered)

for i in range(0,len(MonthlyEvents)):
    if i == 0:
        MonthlyEvents[0] = max(SnowfallEventNumbered[MonthlyLastTimestep[0]:MonthlyLastTimestep[1]])
    else:
        MonthlyEvents[i] = max(SnowfallEventNumbered[MonthlyLastTimestep[i]:MonthlyLastTimestep[i+1]]) - max(SnowfallEventNumbered[MonthlyLastTimestep[i-1]:MonthlyLastTimestep[i]])       

MonthlyEventIndices = np.zeros(7)
for i in range(0,len(MonthlyEventIndices)):
    MonthlyEventIndices[i-1] = sum(MonthlyEvents[0:i])
MonthlyEventIndices[len(MonthlyEventIndices)-1] = sum(MonthlyEvents)

MonthlyEventIndices = MonthlyEventIndices.astype(int)

*Write variables to CSVs*

In [ ]:
with open(Saved_Data_Path + 'EventLengths.csv', 'w') as f:
    write = csv.writer(f)
    write.writerow(EventLengths)

with open(Saved_Data_Path + 'EventTimesStart.csv', 'w') as f:
    write = csv.writer(f)
    write.writerow(EventTimesStart)

with open(Saved_Data_Path + 'EventTimesEnd.csv', 'w') as f:
    write = csv.writer(f)
    write.writerow(EventTimesEnd)
    
with open(Saved_Data_Path + 'SnowfallEventNumbered.csv', 'w') as f:
    write = csv.writer(f)
    write.writerow(SnowfallEventNumbered)

with open(Saved_Data_Path + 'MonthlyEventIndices.csv', 'w') as f:
    write = csv.writer(f)
    write.writerow(MonthlyEventIndices)

*Open CSVs*

In [22]:
with open(Saved_Data_Path + 'EventLengths.csv') as f:
    reader = csv.reader(f)
    EventLengths = list(reader)

EventLengths = EventLengths[0]
EventLengths = [int(i) for i in EventLengths]

with open(Saved_Data_Path + 'EventTimesStart.csv') as f:
    reader = csv.reader(f)
    EventTimesStart = list(reader)

EventTimesStart = EventTimesStart[0]
EventTimesStart = [int(i) for i in EventTimesStart]

with open(Saved_Data_Path + 'EventTimesEnd.csv') as f:
    reader = csv.reader(f)
    EventTimesEnd = list(reader)

EventTimesEnd = EventTimesEnd[0]
EventTimesEnd = [int(i) for i in EventTimesEnd]

with open(Saved_Data_Path + 'SnowfallEventNumbered.csv') as f:
    reader = csv.reader(f)
    SnowfallEventNumbered = list(reader)

SnowfallEventNumbered = SnowfallEventNumbered[0]
SnowfallEventNumbered = [int(float(i)) for i in SnowfallEventNumbered]

with open(Saved_Data_Path + 'MonthlyEventIndices.csv') as f:
    reader = csv.reader(f)
    MonthlyEventIndices = list(reader)

MonthlyEventIndices = MonthlyEventIndices[0]
MonthlyEventIndices = [int(i) for i in MonthlyEventIndices]